In [1]:
# I realize that this was useless because a line integral along a closed path
# in a conservative vector field will always be equal to 0, but I'm leaving it

def wind_coeff(windspeed, winddirection, gpx):
    wind_vec = (cos(winddirection), sin(winddirection))
    dot_prod = 0
    vec_count = 0
    
    # downsample to once per minute
    for track in gpx.tracks:
        for segment in track.segments:
            segment.points = segment.points[0::60]
            
    for track in gpx.tracks:
        for segment in track.segments:
            for (index, thing) in enumerate(segment.points[:-1]): 
                current, next_ = thing, segment.points[index + 1]

                lat_a = current.latitude * (pi / 180)
                lon_a = current.longitude * (pi / 180)
                lat_b = next_.latitude * (pi / 180)
                lon_b = next_.longitude * (pi / 180)
                delta_long = lon_b - lon_a

                x = cos(lat_b) * sin(delta_long)
                y = cos(lat_a) * sin(lat_b) - sin(lat_a) * cos(lat_b) * cos(delta_long)
                bearing = atan2(x, y)
                dot_prod = dot_prod + cos(bearing) * wind_vec[0] + sin(bearing) * wind_vec[1]
                vec_count = vec_count + 1
                
    return (windspeed * dot_prod / vec_count)

In [2]:
email = 'YOUR EMAIL/USERNAME'
password = 'YOUR PASSWORD'

In [3]:
# log in and go to athlete page
import re
import requests
import werkzeug
werkzeug.cached_property = werkzeug.utils.cached_property
from robobrowser import RoboBrowser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
import gpxpy
import gpxpy.gpx
from datetime import datetime
import json
import pytz
import time
from meteostat import Stations, Hourly, Daily
from math import cos, sin, atan2, pi

site = 'https://www.strava.com'
login_ext = '/login'

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(site + login_ext)

# log in
form = driver.find_element_by_id('email')
form.send_keys(email, Keys.TAB, password)
driver.find_element_by_id('login-button').click()

# go to activities page
time.sleep(3)
driver.find_element_by_xpath("/html/body/div[3]/div/div[1]/div/div/div/div[1]/div[1]/ul/li[3]/a/b").click()

# gather all activity links
time.sleep(3)
page_html = driver.page_source
driver.close()
page_soup = soup(page_html)

[WDM] - Looking for [chromedriver 87.0.4280.88 win32] driver in cache 
[WDM] - File found in cache by path [C:\Users\vance\.wdm\drivers\chromedriver\87.0.4280.88\win32\chromedriver.exe]


In [4]:
# find all activity links
activities = page_soup.findAll('td', {'class': 'view-col col-title'})

links = []
for activity in activities:
    links.append(activity.a['href'])

br = RoboBrowser()
br.open(site + login_ext)

form = br.get_form()
form['email'] = email
form['password'] = password
br.submit_form(form)

C:\Users\vance\Anaconda3\lib\site-packages\robobrowser\browser.py:40: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 40 of the file C:\Users\vance\Anaconda3\lib\site-packages\robobrowser\browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  features=self.browser.parser,


In [5]:
# open each activity page
for (index, activity) in enumerate(links):
    br.open(activity)
    description_cont = br.find_all('div', {'class': 'activity-description'})
    description = ""
    
    if len(description_cont) != 0:
        description = description_cont[0].select('div > p')[0].get_text()
        
    if 'WEATHER:\n' not in description:
        # download GPX file
        gpx_url = activity + '/export_gpx'
        
        br.open(gpx_url)
        gpx = str(br.parsed()[0])[12:-15]
        
        # find location
        gpx = gpxpy.parse(gpx)
        lat = gpx.tracks[0].segments[0].points[0].latitude
        lon = gpx.tracks[0].segments[0].points[0].longitude
        time = gpx.tracks[0].segments[0].points[0].time
        
        # get meteostat ID of nearest station
        stations = Stations()
        station_id = stations.nearby(lat, lon).fetch(1).index[0]

        # get hourly weather at nearest station
        time = time.astimezone(pytz.timezone('US/Central'))
        start = datetime(time.year, time.month, time.day, time.hour, time.minute, time.second)

        if time.hour == 23:
            time = time.astimezone(pytz.timezone('US/Eastern'))
        end = datetime(time.year, time.month, time.day, time.hour + 1, time.minute, time.second)

        weather_data = Hourly(station_id, start, end).fetch()

        temperature = round(weather_data['temp'][0] * 9 / 5 + 32)
        humidity = weather_data['rhum'][0]
        windspeed = 0.621 * weather_data['wspd'][0]
        winddirection = weather_data['wdir'][0] * (pi / 180)
        precipitation = weather_data['prcp'][0]
                
        # compute wind coefficient
        wind_co = wind_coeff(windspeed, winddirection, gpx)
        
        # write to description
        weather_report = ''
        
        if description != "":
            weather_report = '\n------------------\n'
            
        weather_report = weather_report + 'WEATHER:\n'
        weather_report = weather_report + 'Temperature: ' + str(temperature) + '°F, '
        weather_report = weather_report + 'Humidity: ' + str(humidity) + '%, '
        weather_report = weather_report + 'Wind: ' + str(round(wind_co)) + ', '
        weather_report = weather_report + 'Precipitation: ' + str(precipitation)
        
        updated_description = description + weather_report
        
        br.open(activity + '/edit')
        
        form = br.get_form(id = 'edit-activity')
        form['activity[description]'] = updated_description
        
        br.submit_form(form)

C:\Users\vance\Anaconda3\lib\site-packages\robobrowser\browser.py:40: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 40 of the file C:\Users\vance\Anaconda3\lib\site-packages\robobrowser\browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  features=self.browser.parser,
